In [1]:
import numpy as np
import open3d as o3d
import copy
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from matplotlib.colors import LinearSegmentedColormap

from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.GeomAbs import GeomAbs_Circle, GeomAbs_Ellipse, GeomAbs_Line, GeomAbs_BSplineCurve, GeomAbs_Hyperbola, GeomAbs_Parabola
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Display.SimpleGui import init_display
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeVertex
from PyQt5 import QtWidgets
from OCC.Core.GCPnts import GCPnts_UniformDeflection
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Read processed point cloud data

point_dictionary_path = '/home/chris/Code/PointClouds/data/hull_pcd/morten_plate/mapped_hull_points_dictionary_morten_full.pkl'

hull_centers_dictionary = {}

with open(point_dictionary_path, 'rb') as f:
    hull_points_dictionary = pickle.load(f)

hull_pcd = o3d.geometry.PointCloud()

for label, points in hull_points_dictionary.items():
    hull_pcd.points.extend(o3d.utility.Vector3dVector(points))
    center = np.mean(points, axis=0)
    hull_centers_dictionary[label] = center
    
hull_pcd.paint_uniform_color([0.1, 1, 0.1])

hull_centers_pcd = o3d.geometry.PointCloud()
hull_centers_points = list(hull_centers_dictionary.values())
hull_centers_pcd.points = o3d.utility.Vector3dVector(np.array(hull_centers_points))
hull_centers_pcd.paint_uniform_color([1, 0, 0])  # Color centers red

print("Point cloud data has been loaded successfully.")

Point cloud data has been loaded successfully.


In [3]:
# # Load cleaned hulls

# hull_pcd = o3d.io.read_point_cloud("/home/chris/Code/PointClouds/data/ply/MortenHulls.ply")

# eps = 2.5
# min_points = 15

# hulls_labels = np.array(hull_pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))

# max_label = hulls_labels.max()
# print(f"Hull point cloud has {max_label + 1} clusters")
# base_cmap = plt.get_cmap("tab20")
# color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
# colors = np.array(color_cycle)[hulls_labels]
# colors[hulls_labels<0] = 0
# hull_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

# hull_points_dictionary = {}
# hull_centers_dictionary = {}

# # Extract the points and calculate the centers for each cluster
# for label in np.unique(hulls_labels):
#     if label != -1:  # Ignore noise
#         cluster_points = np.asarray(hull_pcd.points)[hulls_labels == label]
#         hull_points_dictionary[label] = cluster_points
#         hull_centers_dictionary[label] = np.mean(cluster_points, axis=0)

# hull_centers_pcd = o3d.geometry.PointCloud()
# hull_centers_points = list(hull_centers_dictionary.values())
# hull_centers_pcd.points = o3d.utility.Vector3dVector(np.array(hull_centers_points))
# hull_centers_pcd.paint_uniform_color([0, 1, 0])  # Color centers blue

# o3d.visualization.draw_geometries([hull_pcd])

In [4]:
# Read cad model data (STEP file)

step_file_path = '/home/chris/Code/PointClouds/data/other_files/MortenPartSTEPVersion.STEP'

def load_step_file(file_path):
    step_reader = STEPControl_Reader()
    status = step_reader.ReadFile(file_path)
    if status == IFSelect_RetDone:
        step_reader.TransferRoots()
        shape = step_reader.OneShape()
        return shape
    else:
        raise Exception("Error: Cannot read STEP file.")

shape = load_step_file(step_file_path)

print('Step file has been loaded successfully.')

Step file has been loaded successfully.


In [5]:
# Extract edges at the top Z-coordinate

def extract_top_edges(shape, deflection=0.1):
    edges = []
    max_z = -float('inf')
    edge_z_coordinates = []

    # First pass to collect the highest Z-coordinate of each edge
    exp_edge = TopExp_Explorer(shape, TopAbs_EDGE)
    while exp_edge.More():
        edge = topods.Edge(exp_edge.Current())
        edge_max_z = -float('inf')
        exp_vertex = TopExp_Explorer(edge, TopAbs_VERTEX)
        vertices = []
        while exp_vertex.More():
            vertex = topods.Vertex(exp_vertex.Current())
            point = BRep_Tool.Pnt(vertex)
            vertices.append(point)
            if point.Z() > edge_max_z:
                edge_max_z = point.Z()
            exp_vertex.Next()
        edge_z_coordinates.append((edge, edge_max_z, vertices))
        if edge_max_z > max_z:
            max_z = edge_max_z
        exp_edge.Next()

    #print(f"Total number of edges: {len(edge_z_coordinates)}")
    #print(f"Maximum Z-coordinate found: {max_z}")

    # Collect edges on the top plane
    top_edges = []
    point_cloud = []
    for edge, z, vertices in edge_z_coordinates:
        if all(abs(vertex.Z() - max_z) < 1e-3 for vertex in vertices):  # Ensure all vertices are at the top surface
            top_edges.append(edge)
            #print(f"Top edge added with highest vertex Z: {z}")

            # Sample points along the edge
            curve = BRepAdaptor_Curve(edge)
            u_min, u_max = curve.FirstParameter(), curve.LastParameter()

            # Calculate the length of the edge
            linear_props = GProp_GProps()
            brepgprop.LinearProperties(edge, linear_props)
            length = linear_props.Mass()

            # Calculate the number of samples based on the length and the desired point density
            num_samples = int(length / deflection)

            for i in range(num_samples):
                u = u_min + i * (u_max - u_min) / (num_samples - 1)
                pnt = curve.Value(u)
                point_cloud.append((pnt.X(), pnt.Y(), pnt.Z()))  # Store the coordinates as a tuple

    print(f"Total number of top view edges: {len(top_edges)}")
    print(f"Total number of points in the cad point cloud: {len(point_cloud)}")
    return top_edges, point_cloud

# Visualize the top edges
def visualize_top_edges(top_edges):
    display, start_display, add_menu, add_function_to_menu = init_display()

    # Assign the color red to each top edge
    red_color = Quantity_Color(1.0, 0.0, 0.0, Quantity_TOC_RGB)
    
    # Display each top edge with the red color
    for edge in top_edges:
        display.DisplayShape(edge, update=True, color=red_color)

    start_display()


# Main execution
print('Generating top view edges and sampling to create the associated point cloud...')
top_edges, cad_points = extract_top_edges(shape)
# if top_edges:
#     visualize_top_edges(top_edges)
# else:
#     print("Top edges not found.")

cad_pcd = o3d.geometry.PointCloud()
cad_pcd.points = o3d.utility.Vector3dVector(np.asarray(cad_points))
o3d.visualization.draw_geometries([cad_pcd])

print('Top view cad point cloud created.')

Generating top view edges and sampling to create the associated point cloud...
Total number of top view edges: 586
Total number of points in the cad point cloud: 81774
Top view cad point cloud created.


In [6]:
# Apply clustering to the cad_pcd

eps = 0.50
min_points = 10

cad_shapes_labels = np.array(cad_pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))

max_label = cad_shapes_labels.max()
print(f"Cad point cloud has {max_label + 1} clusters")
base_cmap = plt.get_cmap("tab20")
color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
colors = np.array(color_cycle)[cad_shapes_labels]
colors[cad_shapes_labels<0] = 0
cad_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

cad_points_dictionary = {}
cad_centers_dictionary = {}

# Extract the points and calculate the centers for each cluster
for label in np.unique(cad_shapes_labels):
    if label != -1:  # Ignore noise
        cluster_points = np.asarray(cad_pcd.points)[cad_shapes_labels == label]
        cad_points_dictionary[label] = cluster_points
        cad_centers_dictionary[label] = np.mean(cluster_points, axis=0)

cad_centers_pcd = o3d.geometry.PointCloud()
cad_centers_points = list(cad_centers_dictionary.values())
cad_centers_pcd.points = o3d.utility.Vector3dVector(np.array(cad_centers_points))
cad_centers_pcd.paint_uniform_color([0, 1, 0])  # Color centers blue

#o3d.visualization.draw_geometries([cad_pcd])

Precompute neighbors.[================>           Cad point cloud has 79 clusters
Precompute neighbors.[========================================] 100%
Clustering[========================================] 100%


PointCloud with 79 points.

In [7]:
#Align the centers of the two clouds for an initial alignment

cad_pcd_center = np.mean(np.asarray(cad_pcd.points), axis=0)
hull_pcd_center = np.mean(np.asarray(hull_pcd.points), axis=0)

# Compute the translation vector
translation = cad_pcd_center - hull_pcd_center

# Translate the hull_pcd to align the centers
hull_pcd.points = o3d.utility.Vector3dVector(np.asarray(hull_pcd.points) + translation)

#o3d.visualization.draw_geometries([hull_pcd, cad_pcd])

In [8]:
front =  [0.0, 0.0, 1.0]
lookat = [-105.36407274754953, -106.22557127184305, 2.0]
up =  [0.0, 1.0, 0.0]
zoom = 0.69999999999999996

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([target_temp, source_temp],
                                      zoom=zoom,
                                      front=front,
                                      lookat=lookat,
                                      up=up)
    
source = copy.deepcopy(hull_pcd)
target = copy.deepcopy(cad_pcd)

In [9]:
# Total points alignment: Align point clouds through ICP based on the lowest RMSE (use all points)

def evaluate_icp(source, target, threshold, trans_init, verbose=False):
    # Perform ICP registration
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    
    # Evaluate the registration
    evaluation = o3d.pipelines.registration.evaluate_registration(
        source, target, threshold, reg_p2p.transformation)
    
    if verbose:
        print(f"Threshold: {threshold}")
        print(f"Fitness: {reg_p2p.fitness}")
        print(f"Inlier RMSE: {reg_p2p.inlier_rmse}")
        print(f"Evaluation Fitness: {evaluation.fitness}")
        print(f"Evaluation Inlier RMSE: {evaluation.inlier_rmse}")
        print("Transformation Matrix:")
        print(reg_p2p.transformation)
        print("-" * 40)
    
    return reg_p2p, evaluation.inlier_rmse, threshold

# Initial transformation (identity matrix)
trans_init = np.eye(4)

# Evaluate ICP for different thresholds
thresholds = [0.5, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.67, 0.69, 0.7, 0.71, 0.72, 0.73, 0.8, 0.85, 0.9, 0.95, 1, 1.1, 1.2, 1.25, 1.3, 1,325, 1.33, 1.34, 1.35, 1.36, 1.37, 1.375, 1.4, 1.425, 1.45, 1.475, 1.5, 2, 3]
results = [evaluate_icp(source, target, threshold, trans_init, verbose=False) for threshold in tqdm(thresholds)]
best_result = min(results, key=lambda x: x[1])

print(f'Best threshold for total alignment was {best_result[2]}')

# Visual inspection for the best threshold 
draw_registration_result(source, target, best_result[0].transformation)

100%|██████████| 43/43 [00:08<00:00,  5.02it/s]


Best threshold for total alignment was 0.58


In [10]:
# Create the transformed hull_dictionary

transformation_matrix = best_result[0].transformation
def transform_points(points, transformation_matrix):
    # Convert points to homogeneous coordinates
    ones_column = np.ones((points.shape[0], 1))
    points_homogeneous = np.hstack((points, ones_column))
    # Apply the transformation matrix
    transformed_points_homogeneous = points_homogeneous.dot(transformation_matrix.T)
    # Convert back to Cartesian coordinates
    return transformed_points_homogeneous[:, :3]

transformed_hull_points_dictionary = {}

for label, points in hull_points_dictionary.items():
    points = points + translation
    transformed_points = transform_points(np.array(points), transformation_matrix)
    transformed_hull_points_dictionary[label] = transformed_points

transformed_hull_pcd = o3d.geometry.PointCloud()

hull_points_indices = {}
index = 0

for labels, points in transformed_hull_points_dictionary.items():
    transformed_hull_pcd.points.extend(o3d.utility.Vector3dVector(points))

transformed_hull_pcd.paint_uniform_color([0.1, 1, 0.1])

#o3d.visualization.draw_geometries([cad_pcd, transformed_hull_pcd],zoom=zoom,front=front,lookat=lookat,up=up)

PointCloud with 29293 points.

In [11]:
# Match shapes through nearest neighbor voting

cad_points = np.asarray(cad_pcd.points)
cad_kdtree = cKDTree(cad_points)

matching_labels = []
for label, points in tqdm(transformed_hull_points_dictionary.items()):
    cad_clusters_votes_dictionary = {}
    for point in points:
        _, index = cad_kdtree.query(point)
        cluster_label = cad_shapes_labels[index]
        if cluster_label in cad_clusters_votes_dictionary:
            cad_clusters_votes_dictionary[cluster_label] += 1
        else:
            cad_clusters_votes_dictionary[cluster_label] = 1
    chosen_cad_cluster_label = max(cad_clusters_votes_dictionary, key=cad_clusters_votes_dictionary.get)
    matching_labels.append((label,chosen_cad_cluster_label))

print(matching_labels)

  0%|          | 0/81 [00:00<?, ?it/s]

100%|██████████| 81/81 [00:01<00:00, 50.21it/s] 

[(1, 40), (2, 55), (3, 41), (4, 42), (5, 76), (6, 54), (7, 50), (8, 46), (9, 75), (10, 49), (11, 53), (12, 45), (13, 74), (14, 48), (15, 52), (16, 44), (17, 37), (18, 51), (19, 47), (20, 43), (21, 73), (22, 58), (23, 6), (24, 5), (25, 4), (26, 3), (27, 72), (28, 72), (29, 39), (30, 72), (31, 71), (32, 36), (33, 30), (34, 2), (35, 24), (36, 18), (37, 12), (38, 61), (39, 70), (40, 77), (41, 38), (42, 69), (43, 35), (44, 29), (45, 23), (46, 17), (47, 11), (48, 68), (49, 1), (50, 67), (51, 34), (52, 28), (53, 22), (54, 16), (55, 10), (56, 66), (57, 27), (58, 33), (59, 21), (60, 65), (61, 15), (62, 9), (63, 78), (64, 0), (65, 64), (66, 32), (67, 26), (68, 20), (69, 14), (70, 8), (71, 63), (72, 62), (73, 31), (74, 25), (75, 19), (76, 13), (77, 7), (78, 60), (79, 57), (80, 56), (81, 59)]


In [12]:
# Visualize the matching shapes

colormap = plt.get_cmap('tab20')
colors = [colormap(i % 20) for i in range(len(matching_labels))]

# Create a new point cloud for visualization
hull_pcd_colored = o3d.geometry.PointCloud()
cad_pcd_colored = o3d.geometry.PointCloud()

# Assign colors to the points based on the matching labels
for i, (hull_label, cad_label) in enumerate(matching_labels):
    hull_color = colors[i][:3]
    cad_color = colors[i][:3]

    hull_points = transformed_hull_points_dictionary[hull_label]
    cad_points = cad_points_dictionary[cad_label]

    hull_pcd_colored.points.extend(o3d.utility.Vector3dVector(hull_points))
    cad_pcd_colored.points.extend(o3d.utility.Vector3dVector(cad_points))

    hull_pcd_colored.colors.extend(o3d.utility.Vector3dVector([hull_color] * len(hull_points)))
    cad_pcd_colored.colors.extend(o3d.utility.Vector3dVector([cad_color] * len(cad_points)))

# Visualize the colored point clouds
o3d.visualization.draw_geometries([hull_pcd_colored, cad_pcd_colored])

In [13]:
# Calculate distances/errors for each pair

distance_dictionary = {}

for hull_label, cad_label in matching_labels:
    hull_cluster_points = transformed_hull_points_dictionary[hull_label]
    cad_cluster_points = cad_points_dictionary[cad_label]
    cad_cluster_kdtree = cKDTree(cad_cluster_points)
    distances, _ = cad_cluster_kdtree.query(hull_cluster_points)
    average_distance = np.mean(distances)
    distance_dictionary[hull_label] = average_distance

print(distance_dictionary)

{1: 0.1651834536759075, 2: 0.3052723058956192, 3: 0.14321607764214228, 4: 0.2700913574219084, 5: 0.11443174309576813, 6: 0.24070777981766334, 7: 0.15859269677722038, 8: 0.16003581309390147, 9: 0.1963864928186037, 10: 0.13506477715176143, 11: 0.17869061053122898, 12: 0.13943249678726627, 13: 0.09887953322054364, 14: 0.16609051642745895, 15: 0.13611442740547502, 16: 0.17370453736069313, 17: 0.18528356091974918, 18: 0.1325241492650887, 19: 0.1272036434351676, 20: 0.1973206312584251, 21: 0.16062156366706606, 22: 0.08040022105070663, 23: 0.11196755798535316, 24: 0.1544777124075152, 25: 0.12250002856329972, 26: 0.13619228084779342, 27: 2.0790895856664817, 28: 1.9552690803576036, 29: 0.12670244587862545, 30: 0.08091304785455299, 31: 0.09795789066582242, 32: 0.08649940714977564, 33: 0.20076785647827505, 34: 0.121527113403201, 35: 0.11690844889621677, 36: 0.09227948026018867, 37: 0.10550442164520141, 38: 0.09543791929835992, 39: 0.2309107736814994, 40: 0.2152880093069456, 41: 0.0862036335679579

In [14]:
# Color the errors

colors = ["green", "yellow", "red"]
n_bins = 10
cmap = LinearSegmentedColormap.from_list("custom_green_yellow_red", colors, N=n_bins)
error_color_list = [cmap(i/n_bins)[:3] for i in range(n_bins)]

print(error_color_list)

error_values = [error for _, error in distance_dictionary.items()]
min_error, max_error = min(error_values), max(error_values)
step = (max_error - min_error) / n_bins if error_values else 0

transformed_hull_pcd_points = np.asarray(transformed_hull_pcd.points)
transformed_hull_pcd_colors = np.zeros_like(transformed_hull_pcd_points)

for label, hull_cluster_points in transformed_hull_points_dictionary.items():
    error = distance_dictionary[label]
    percentile = int((error - min_error) // step)
    percentile = min(percentile, n_bins - 1)
    color = error_color_list[percentile]

    for point in hull_cluster_points:
        index = np.where((transformed_hull_pcd_points == point).all(axis=1))[0]
        transformed_hull_pcd_colors[index] = color

transformed_hull_pcd.colors = o3d.utility.Vector3dVector(transformed_hull_pcd_colors)
cad_pcd.paint_uniform_color([0.6,0.6,0.6])

[(0.0, 0.5019607843137255, 0.0), (0.2222222222222222, 0.612636165577342, 0.0), (0.4444444444444444, 0.7233115468409586, 0.0), (0.6666666666666666, 0.8339869281045751, 0.0), (0.8888888888888888, 0.9446623093681916, 0.0), (1.0, 0.8888888888888888, 0.0), (1.0, 0.6666666666666667, 0.0), (1.0, 0.44444444444444464, 0.0), (1.0, 0.2222222222222222, 0.0), (1.0, 0.0, 0.0)]


PointCloud with 81774 points.

In [15]:
# Draw final output

o3d.visualization.draw_geometries([cad_pcd, transformed_hull_pcd])

In [16]:
# Calculate local errors

cad_points = np.asarray(cad_pcd.points)
cad_kdtree = cKDTree(cad_points)

local_errors = []

for point in transformed_hull_pcd.points:
    distance, _ = cad_kdtree.query(point)
    local_errors.append(distance)

local_errors = np.array(local_errors)

# Create colormap for visualization
colors = ["green", "yellow", "red"]
n_bins = 10
cmap = LinearSegmentedColormap.from_list("custom_green_yellow_red", colors, N=n_bins)

# Normalize the errors to the range [0, 1]
min_error, max_error = local_errors.min(), local_errors.max()
norm_errors = (local_errors - min_error) / (max_error - min_error)

# Map normalized errors to colors
mapped_colors = cmap(norm_errors)

# Assign colors to the point cloud
transformed_hull_pcd.colors = o3d.utility.Vector3dVector(mapped_colors[:, :3])

# Visualize the point clouds
cad_pcd.paint_uniform_color([0.3, 0.3, 0.3])
o3d.visualization.draw_geometries([cad_pcd, transformed_hull_pcd])

In [17]:
import pandas as pd

def create_error_table(matching_labels, transformed_hull_points_dictionary, cad_points_dictionary, distance_threshold=0.2):
    error_table = []
    for hull_label, cad_label in matching_labels:
        hull_cluster_points = transformed_hull_points_dictionary[hull_label]
        cad_cluster_points = cad_points_dictionary[cad_label]
        cad_cluster_kdtree = cKDTree(cad_cluster_points)
        distances, _ = cad_cluster_kdtree.query(hull_cluster_points)
        
        # Count points exceeding the threshold
        exceed_threshold_count = np.sum(distances > distance_threshold)
        
        error_table.append({
            'Hull Label': hull_label,
            'CAD Label': cad_label,
            'Total Points': len(hull_cluster_points),
            'Points Exceeding Threshold': exceed_threshold_count
        })
    
    error_df = pd.DataFrame(error_table)
    return error_df


In [18]:
# Create the error table for points exceeding 0.2mm threshold
error_threshold = 0.45
error_table = create_error_table(matching_labels, transformed_hull_points_dictionary, cad_points_dictionary, distance_threshold=error_threshold)

# Print the error table
print(error_table.to_string(index=False))

# After the local errors are calculated
local_errors = np.array(local_errors)

# Create the error table for points exceeding 0.2mm threshold
error_threshold = 0.45
error_table = create_error_table(matching_labels, transformed_hull_points_dictionary, cad_points_dictionary, distance_threshold=error_threshold)

# Print the error table
print(error_table.to_string(index=False))

# Create colormap for visualization
colors = ["green", "yellow", "red"]
n_bins = 10
cmap = LinearSegmentedColormap.from_list("custom_green_yellow_red", colors, N=n_bins)

# Normalize the errors to the range [0, 1]
min_error, max_error = local_errors.min(), local_errors.max()
norm_errors = (local_errors - min_error) / (max_error - min_error)

# Map normalized errors to colors
mapped_colors = cmap(norm_errors)

# Assign colors to the point cloud
transformed_hull_pcd.colors = o3d.utility.Vector3dVector(mapped_colors[:, :3])

# Visualize the point clouds
cad_pcd.paint_uniform_color([0.3, 0.3, 0.3])
o3d.visualization.draw_geometries([cad_pcd, transformed_hull_pcd])

 Hull Label  CAD Label  Total Points  Points Exceeding Threshold
          1         40           298                           0
          2         55            17                           5
          3         41            17                           0
          4         42            17                           5
          5         76           442                           0
          6         54            17                           0
          7         50            21                           0
          8         46            21                           0
          9         75           656                           0
         10         49            17                           0
         11         53            25                           0
         12         45            23                           0
         13         74           447                           0
         14         48            17                           0
         15         52   

In [19]:
# List of Hull and CAD labels to visualize
hull_labels_to_keep = [9, 39, 40, 57, 63, 64, 65, 71, 78, 81]
cad_labels_to_keep = [75, 70, 77, 27, 78, 0, 64, 63, 60, 59]

In [20]:
hull_labels_to_keep = [64, 65, 81]
cad_labels_to_keep = [0, 64, 59]

In [21]:
# Filter the transformed hull point cloud
filtered_hull_pcd = o3d.geometry.PointCloud()
for label in hull_labels_to_keep:
    if label in transformed_hull_points_dictionary:
        points = transformed_hull_points_dictionary[label]
        filtered_hull_pcd.points.extend(o3d.utility.Vector3dVector(points))

# Filter the CAD point cloud
filtered_cad_pcd = o3d.geometry.PointCloud()
for label in cad_labels_to_keep:
    if label in cad_points_dictionary:
        points = cad_points_dictionary[label]
        filtered_cad_pcd.points.extend(o3d.utility.Vector3dVector(points))

In [22]:
# Visualize the filtered point clouds
o3d.visualization.draw_geometries([filtered_hull_pcd, filtered_cad_pcd])